In [88]:
'''

Much of this code is adapted from code written by 
Xifeng Guo, E-mail: `guoxifeng1990@163.com`, Github: `https://github.com/XifengGuo/CapsNet-Keras`
'''


import keras.backend as K
import tensorflow as tf
from keras import initializers, layers, activations


class Length(layers.Layer):
    """
    Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss.
    Using this layer as model's output can directly predict labels by using `y_pred = np.argmax(model.predict(x), 1)`
    inputs: shape=[None, num_vectors, dim_vector]
    output: shape=[None, num_vectors]
    
    Author: Xifeng Guo, E-mail: `guoxifeng1990@163.com`, Github: `https://github.com/XifengGuo/CapsNet-Keras`
    """
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]


def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

class CapsuleLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_capsule] and output shape = \
    [None, num_capsule, dim_capsule]. For Dense Layer, input_dim_capsule = dim_capsule = 1.
    
    :param num_capsule: number of capsules in this layer
    :param dim_capsule: dimension of the output vectors of the capsules in this layer
    :param num_routing: number of iterations for the routing algorithm

    Author: Xifeng Guo, E-mail: `guoxifeng1990@163.com`, Github: `https://github.com/XifengGuo/CapsNet-Keras`
    """
    def __init__(self, num_capsule, dim_capsule, num_routing=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.num_routing = num_routing
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_capsule]
        # inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
        inputs_expand = K.expand_dims(inputs, 1)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # inputs_tiled.shape=[None, num_capsule, input_num_capsule, input_dim_capsule]
        inputs_tiled = K.tile(inputs_expand, [1, self.num_capsule, 1, 1])

        # Compute `inputs * W` by scanning inputs_tiled on dimension 0.
        # x.shape=[num_capsule, input_num_capsule, input_dim_capsule]
        # W.shape=[num_capsule, input_num_capsule, dim_capsule, input_dim_capsule]
        # Regard the first two dimensions as `batch` dimension,
        # then matmul: [input_dim_capsule] x [dim_capsule, input_dim_capsule]^T -> [dim_capsule].
        # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
        inputs_hat = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs_tiled)

        """
        # Begin: routing algorithm V1, dynamic ------------------------------------------------------------#
        # The prior for coupling coefficient, initialized as zeros.
        b = K.zeros(shape=[self.batch_size, self.num_capsule, self.input_num_capsule])

        def body(i, b, outputs):
            c = tf.nn.softmax(b, dim=1)  # dim=2 is the num_capsule dimension
            outputs = squash(K.batch_dot(c, inputs_hat, [2, 2]))
            if i != 1:
                b = b + K.batch_dot(outputs, inputs_hat, [2, 3])
            return [i-1, b, outputs]

        cond = lambda i, b, inputs_hat: i > 0
        loop_vars = [K.constant(self.num_routing), b, K.sum(inputs_hat, 2, keepdims=False)]
        shape_invariants = [tf.TensorShape([]),
                            tf.TensorShape([None, self.num_capsule, self.input_num_capsule]),
                            tf.TensorShape([None, self.num_capsule, self.dim_capsule])]
        _, _, outputs = tf.while_loop(cond, body, loop_vars, shape_invariants)
        # End: routing algorithm V1, dynamic ------------------------------------------------------------#
        """
        # Begin: Routing algorithm ---------------------------------------------------------------------#
        # In forward pass, `inputs_hat_stopped` = `inputs_hat`;
        # In backward, no gradient can flow from `inputs_hat_stopped` back to `inputs_hat`.
        inputs_hat_stopped = K.stop_gradient(inputs_hat)
        
        # The prior for coupling coefficient, initialized as zeros.
        # b.shape = [None, self.num_capsule, self.input_num_capsule].
        b = tf.zeros(shape=[K.shape(inputs_hat)[0], self.num_capsule, self.input_num_capsule])

        assert self.num_routing > 0, 'The num_routing should be > 0.'
        for i in range(self.num_routing):
            # c.shape=[batch_size, num_capsule, input_num_capsule]
            c = tf.nn.softmax(b, dim=1)

            # At last iteration, use `inputs_hat` to compute `outputs` in order to backpropagate gradient
            if i == self.num_routing - 1:
                # c.shape =  [batch_size, num_capsule, input_num_capsule]
                # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
                # The first two dimensions as `batch` dimension,
                # then matmal: [input_num_capsule] x [input_num_capsule, dim_capsule] -> [dim_capsule].
                # outputs.shape=[None, num_capsule, dim_capsule]
                outputs = squash(K.batch_dot(c, inputs_hat, [2, 2]))  # [None, 10, 16]
            else:  # Otherwise, use `inputs_hat_stopped` to update `b`. No gradients flow on this path.
                outputs = squash(K.batch_dot(c, inputs_hat_stopped, [2, 2]))

                # outputs.shape =  [None, num_capsule, dim_capsule]
                # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
                # The first two dimensions as `batch` dimension,
                # then matmal: [dim_capsule] x [input_num_capsule, dim_capsule]^T -> [input_num_capsule].
                # b.shape=[batch_size, num_capsule, input_num_capsule]
                b += K.batch_dot(outputs, inputs_hat_stopped, [2, 3])
        # End: Routing algorithm -----------------------------------------------------------------------#

        return outputs

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])


def PrimaryCap(inputs, L_hat, dim_capsule, n_channels, kernel_size, activation):
    """
    Apply GraphConv `n_channels` times and concatenate all capsules
    :param inputs: 3D tensor, shape=[None, N, channels]
    :param dim_capsule: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_capsule]

    Author: David McDonald, E-mail: `dxm237@cs.bham.ac.uk`
    """
    output = GraphConv(L_hat=L_hat, num_filters=dim_capsule*n_channels, kernel_size=kernel_size, activation=activation,
                           name='primarycap_GraphConv')(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)

class GraphConv(layers.Layer):
    """
    A layer to perform a convolutional filter on a graph

    :param L_hat: rescaled graph laplacian: L_hat = 2L/lambda_max - Identity(N)
    :param num_filters: number of filters
    :param kernel_size: size of kernel to use
    
    Author: David McDonald, Email: `dxm237@cs.bham.ac.uk'
    """
    def __init__(self, L_hat, num_filters, kernel_size, activation=None,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(GraphConv, self).__init__(**kwargs)
        assert len(L_hat.shape) == 3, "L_hat must have three dimensions: [1, N, N]"
        self.L_hat = L_hat
#         assert num_filters > 3, "number of filters must be greater than 3"
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.activation = activation
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, N, input_channels]"
        # self.input_num_capsule = input_shape[1]
        # self.input_dim_capsule = input_shape[2]
        '''
        input shape = [None, N, input_channels]
        '''
        self.input_channels = input_shape[2]
        
        # chebyshev co-efficients
        self.W = self.add_weight(shape=[self.input_channels * (self.kernel_size + 1), self.num_filters],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True
        
    def compute_x_hat(self, x):
        '''
        efficient computation of filter using recurrence relation of Chebyshev polynomials
        T_k(x) = 2xT_k-1(x) - T_k-2(x)
        T_0(x) = 1
        T_1(x) = x
        
        input shape is [None, N, input_channels]
        output shape is [None, N, filters]
        '''
        # tile L_hat batch_size times
        L_hat = K.tile(self.L_hat, [K.shape(x)[0], 1, 1])
        # batch_dot(L, x) shape is [None, N, channels]
        x_hat = [x]
        if self.kernel_size > 0:
            x_hat.append(K.batch_dot(L_hat, x))
        
        for k in range(self.kernel_size - 1):
            x_hat.append(2 * K.batch_dot(L_hat, x_hat[-1]) - x_hat[-2])
#         print K.shape(K.concatenate(x_hat, axis=-1))
        
        # concatenate to combine input_channels and kernel_size axes
        return K.concatenate(x_hat, axis=-1)
        

    def call(self, inputs):

        '''
        chebyshev polynomial for filtering using coefficients started in kernel
        '''
        x_hat = self.compute_x_hat(inputs)
        # x_hat shape = [None, N, input_channels*filter_size]
        # W shape = [input_channels*filter_size, num_filters]
        # output shape = [None, N, num_filters]
        output = K.dot(x_hat, self.W)

        if self.activation is not None:
            output = activations.get(self.activation)(output)
        
        return output
        
    def compute_output_shape(self, input_shape):
        '''
        output_shape is [None, N, num_filters]
        '''
        return tuple([None, input_shape[1], self.num_filters])

In [20]:
import numpy as np
import scipy as sp
import pandas as pd

from keras.models import Model
from keras.layers import Input

import networkx as nx

In [21]:
memberships = pd.read_csv("../data/0_facebook_infomap.csv", index_col=0, header=0).to_dict()["x"]
memberships = {k-1 : v for k, v in memberships.items()}

In [22]:
G = nx.read_gml("../data/0_facebook_graph.gml")
nx.set_node_attributes(G, name="membership", values=memberships)
G = max(nx.connected_component_subgraphs(G), key=len)

In [23]:
N = nx.number_of_nodes(G)
num_channels = 1

In [24]:
mems = nx.get_node_attributes(G, "membership").values()
mems = np.array(mems).reshape(1, N, 1)
mems = (mems - mems.mean()) / mems.std()

In [83]:
L = nx.laplacian_matrix(G).astype(np.float32)
l, U = sp.sparse.linalg.eigen.arpack.eigsh(L, k=1, which="LM")

L_hat = 2 * L / l[0] - sp.sparse.identity(N, dtype=np.float32)

L_hat_tensor = K.constant(L_hat.todense(), dtype=K.floatx())
L_hat_tensor = K.expand_dims(L_hat_tensor, 0)

In [160]:
x = Input(shape=(N, num_channels), name="input_layer")

conv1 = GraphConv(L_hat=L_hat_tensor, num_filters=5, kernel_size=1, activation="relu")(x)

primary_cap = PrimaryCap(inputs=conv1, L_hat=L_hat_tensor, dim_capsule=8, n_channels=16, kernel_size=1, activation="relu")

secondary_cap = CapsuleLayer(dim_capsule=16, num_capsule=32, num_routing=3)(primary_cap)

secondary_cap_length = Length()(secondary_cap)

dense = layers.Dense(10, activation="relu", name="dense_layer")(secondary_cap_length)

output = layers.Dense(1, activation="sigmoid", name="output")(dense)

model = Model(x, output)

model.compile(optimizer="adam", loss="binary_crossentropy")

In [161]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 324, 1)            0         
_________________________________________________________________
graph_conv_17 (GraphConv)    (None, 324, 5)            10        
_________________________________________________________________
primarycap_GraphConv (GraphC (None, 324, 128)          1280      
_________________________________________________________________
primarycap_reshape (Reshape) (None, 5184, 8)           0         
_________________________________________________________________
primarycap_squash (Lambda)   (None, 5184, 8)           0         
_________________________________________________________________
capsule_layer_6 (CapsuleLaye (None, 32, 16)            21233664  
_________________________________________________________________
length_6 (Length)            (None, 32)                0         
__________

In [162]:
S = 10000

In [163]:
Y = np.random.randint(2, size=(S, 1),)
X = np.random.normal(loc=mems, scale=0.1, size=(S, N, 1), ) + \
np.random.normal(loc=(Y / 10).reshape(S, 1, 1), scale=0.1, size=(S, N, 1), ) +\
np.random.normal(loc=0, scale=0.01, size=(S, N, 1), )

In [164]:
model.predict(X)

array([[ 0.50000006],
       [ 0.50000006],
       [ 0.50000012],
       ..., 
       [ 0.50000012],
       [ 0.50000006],
       [ 0.50000012]], dtype=float32)

In [159]:
model.fit(X, Y, batch_size=100, epochs=1, verbose=1)

Epoch 1/1
 1900/10000 [====>.........................] - ETA: 3:31 - loss: 0.6933

KeyboardInterrupt: 